In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-15"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14306,2024-03-15,Eua Nba,22:30,Utah Jazz,Atlanta Hawks,1.78,1.88,227.5,1.93,1.93,-2.5,1.97,2.28,QcI5xuFa,0.561798,0.531915,0.518135,0.518135,0.093713,0.4,0.2,NaN,NaN,244.672,69.649810,0.284666,204.056,29.601725,0.145067,230.378,249.488,200.07,157.41,0.0,0.0,0.0,0.0,0.038640,0.000000,0.103154,-1.09,-0.218,-3.577982,0.541306,0.5,-0.041306,-1.90,-0.380,-2.315789,0.454016,0.3,-0.154016
14307,2024-03-15,Europa Euroliga,16:00,Lyon-Villeurbanne,Anadolu Efes,2.88,1.45,169.5,1.91,1.91,4.5,2.00,1.20,KCFiHtnj,0.347222,0.689655,0.523560,0.523560,0.036877,0.4,0.6,NaN,NaN,217.778,104.572337,0.480179,137.202,28.564131,0.208190,157.752,143.694,233.28,136.80,0.0,0.0,0.0,0.0,0.467050,0.000000,0.353553,1.73,0.346,5.433526,0.590892,0.7,0.109108,-2.32,-0.464,-0.969828,0.525691,0.4,-0.125691
14308,2024-03-15,Europa Euroliga,16:30,Bayern,Zalgiris Kaunas,1.61,2.42,159.5,1.91,1.91,-4.5,2.00,3.40,YFJeG01d,0.621118,0.413223,0.523560,0.523560,0.034341,0.6,0.8,NaN,NaN,173.378,79.419596,0.458072,213.568,56.978665,0.266794,132.720,260.366,113.96,245.52,0.0,0.0,0.0,0.0,0.284246,0.000000,0.366648,0.69,0.138,4.420290,0.742571,0.7,-0.042571,0.88,0.176,8.068182,0.330577,0.3,-0.030577
14309,2024-03-15,Europa Euroliga,16:30,Virtus Bologna,Real Madrid,2.51,1.62,163.5,1.85,2.09,3.5,1.97,1.33,zLcvupvc,0.398406,0.617284,0.540541,0.478469,0.015690,0.4,0.4,NaN,NaN,169.618,51.006489,0.300714,111.512,15.387726,0.137992,132.276,122.832,159.90,113.24,0.0,0.0,0.0,0.0,0.304758,0.086145,0.274272,1.36,0.272,5.551471,0.591193,0.9,0.308807,-2.21,-0.442,-1.402715,0.639412,0.6,-0.039412
14310,2024-03-15,China Cba,08:35,Guangdong,Liaoning,2.32,1.58,212.5,1.82,1.92,2.5,1.99,1.39,2RP8qubt,0.431034,0.632911,0.549451,0.520833,0.063946,0.6,0.6,NaN,NaN,196.304,69.663568,0.354876,129.684,17.171503,0.132410,177.940,168.390,306.22,112.36,0.0,0.0,0.0,0.0,0.268338,0.037813,0.251044,-2.65,-0.530,-2.490566,0.740177,0.6,-0.140177,1.11,0.222,2.612613,0.720169,0.7,-0.020169
14311,2024-03-15,China Cba,08:35,Tianjin,Zhejiang Guangsha,2.51,1.50,206.5,1.82,1.92,2.5,2.05,1.34,AgKhw4y7,0.398406,0.666667,0.549451,0.520833,0.065073,0.8,0.2,NaN,NaN,306.144,247.970172,0.809979,137.052,16.976595,0.123870,233.482,141.564,172.36,115.34,0.0,0.0,0.0,0.0,0.356198,0.037813,0.296192,1.65,0.330,4.575758,0.560255,0.8,0.239745,-2.88,-0.576,-0.868056,0.623115,0.2,-0.423115
14312,2024-03-15,Eua Ncaa,00:30,Boise State,New Mexico,1.87,1.95,153.5,1.91,1.91,-2.5,2.05,2.20,IXnyluxe,0.534759,0.512821,0.523560,0.523560,0.047580,0.8,1.0,NaN,NaN,159.704,70.795039,0.443289,107.570,33.855810,0.314733,105.408,124.510,93.06,166.00,0.0,0.0,0.0,0.0,0.029617,0.000000,0.049913,-2.79,-0.558,-1.559140,0.780922,0.7,-0.080922,0.62,0.124,7.661290,0.709238,0.8,0.090762
14313,2024-03-15,Eua Ncaa,00:30,Long Beach State,UC Riverside,1.67,2.28,149.5,1.80,1.95,-4.5,2.05,3.20,vslXlLMl,0.598802,0.438596,0.555556,0.512821,0.037399,0.6,0.6,NaN,NaN,168.230,71.074906,0.422487,148.160,39.303506,0.265277,128.746,163.020,134.94,213.84,0.0,0.0,0.0,0.0,0.218398,0.056569,0.309780,-1.97,-0.394,-1.700508,0.000000,0.0,0.000000,3.76,0.752,1.702128,0.411314,0.4,-0.011314
14314,2024-03-15,Eua Ncaa,00:30,Texas Arlington,Stephen F. Austin,1.67,2.28,141.5,1.91,1.91,-4.5,2.05,3.20,xllQueDo,0.598802,0.438596,0.523560,0.523560,0.037399,0.6,0.2,NaN,NaN,124.654,22.234286,0.178368,116.944,21.852530,0.186863,127.420,120.708,110.70,118.42,0.0,0.0,0.0,0.0,0.218398,0.000000,0.309780,-0.96,-0.192,-3.489583,0

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,22:30,Eua Nba,Utah Jazz,Atlanta Hawks,227.5,1.93,1.0,Over
1,16:30,Europa Euroliga,Bayern,Zalgiris Kaunas,159.5,1.91,1.0,Over
2,00:30,Eua Ncaa,Boise State,New Mexico,153.5,1.91,1.0,Over
3,00:30,Eua Ncaa,Long Beach State,UC Riverside,149.5,1.80,1.0,Over
4,00:30,Eua Ncaa,Texas Arlington,Stephen F. Austin,141.5,1.91,1.0,Over
5,13:30,Finlândia Korisliiga,Bisons Loimaa,Pyrinto Tampere,169.5,1.88,1.0,Over
6,16:00,França Lnb,Roanne,Gravelines-Dunkerque,160.5,1.88,1.0,Over
7,16:00,França Lfb Feminina,La Roche F,Tarbes GB F,141.5,1.81,1.0,Over
8,15:30,República Tcheca 1. Liga,Opava 2,Litomerice,153.5,1.86,1.0,Over
9,14:00,Turquia Tbl,Kocaeli BŞB Kağıtspor,Final Spor,159.5,1.87,1.0,Over
